In [ ]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Bing
from pyproj import Transformer

# requesting Bing to convert latitude and longitude into location
geolocator = Bing(api_key='your_Bing_api_key')

# convert XY map coordinates into longitude into location coordinates
transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

# count time
def display_time(func):
    def wrapper(*args):
        t1 = time.time()
        result = func(*args)
        t2 = time.time()
        print('Total time: {:.4} m'.format((t2 - t1) / 60))
        return result
    return wrapper


# add id and postcode columns to dataframe
def add_id_and_postcode_cols(data):
    '''
    params:
        data: dataframe
    return:
        dataframe in column order 'id', 'stop_id','postcode', 
           'stop_name', 'stop_lat', 'stop_lon', 'mode', 'state_code'
    '''
    data['postcode'] = ''
    data['id'] = range(1, len(data)+1)
    data.columns = data.columns.to_series().apply(lambda x:x.strip()).apply(lambda x:x.lower())
    data = data.rename(columns={'state code':'state_code'})
    cols = ['id', 'stop_id','postcode', 'stop_name', 'stop_lat', 'stop_lon', 'mode', 'state_code']
    data = data[cols]
    data = data.reindex(columns=cols)
    return data


# requesting Bing server to convert lat and lon coordinates, and exctract postcode
@display_time
def get_postcode(data):
    '''
    params:
        data: dataframe
    return:
        dataframe which filled with postcode according to the latitude and longitude
    '''
    for i in range(len(data)):
        coord = str(data['stop_lat'].values[i]) + ' ' + str(data['stop_lon'].values[i])
        data['postcode'].values[i] = geolocator.reverse(coord).address.split(',')[-2].split(' ')[-1]
        # data['postcode'].values[i] = geolocator.reverse(coord).address.split(',')[-2]
    return data

## ACT_BUS

In [ ]:
ACT_BUS = pd.read_csv('dataset/ACT_Bus_Stops_v 2.csv')
ACT_BUS.head(2)

In [ ]:
ACT_BUS[ACT_BUS.duplicated()].count()

In [ ]:
ACT_BUS['id'] = range(1, len(ACT_BUS)+1)

In [ ]:
ACT_BUS['postcode'] = ''

In [ ]:
cols = list(ACT_BUS.columns)
cols.insert(0, cols.pop(-2))
cols.insert(2, cols.pop(-1))

In [ ]:
ACT_BUS = ACT_BUS.reindex(columns=cols).rename(columns={'STATE CODE':'state_code'})

In [ ]:
ACT_BUS.head(2)

In [ ]:
ACT_BUS = get_postcode(ACT_BUS.iloc[:,:])

In [ ]:
ACT_BUS.to_csv('finished/ACT_BUS.csv', index=False)

## VIC_Railway_Station

In [ ]:
VIC_Railway_Station = pd.read_csv('dataset/VIC_Railway Station.csv')
VIC_Railway_Station.head(2)

In [ ]:
VIC_Railway_Station.info()

In [ ]:
VIC_Railway_Station[VIC_Railway_Station.duplicated()].count()

In [ ]:
VIC_Railway_Station[VIC_Railway_Station.duplicated()]

In [ ]:
VIC_Railway_Station = VIC_Railway_Station.drop_duplicates()

In [ ]:
VIC_Railway_Station[VIC_Railway_Station.duplicated()]

In [ ]:
VIC_Railway_Station = add_id_and_postcode_cols(VIC_Railway_Station)
VIC_Railway_Station.head(2)

In [ ]:
VIC_Railway_Station = get_postcode(VIC_Railway_Station.iloc[:,:])

In [ ]:
VIC_Railway_Station.head(2)

In [ ]:
VIC_Railway_Station.to_csv('dataset/finished/VIC_Railway.csv', index=False)

## WA_PT_Autority_Service_Routes

In [ ]:
WA_PT = pd.read_csv('dataset/WA_PT Authority Service Routes (PTA-002).csv')
WA_PT.info()

In [ ]:
WA_PT[WA_PT.duplicated()].count()

In [ ]:
WA_PT[WA_PT['state code'].isna()] = WA_PT[WA_PT['state code'].isna()].fillna('WA')
WA_PT.info()

In [ ]:
WA_PT = add_id_and_postcode_cols(WA_PT)
WA_PT.head(2)

In [ ]:
WA_PT = get_postcode(WA_PT)

In [ ]:
WA_PT.to_csv('dataset/finished/WA_Bus.csv', index=False)

## VIC_Brisbane_City_Council_Bus_Stop

In [ ]:
VIC_Brisbane_Bus = pd.read_csv('dataset/VIC_Brisbane City Council_bus-stop-locations v2.csv', dtype={'stop_id':str})
VIC_Brisbane_Bus.head(2)

In [ ]:
VIC_Brisbane_Bus.info()

In [ ]:
VIC_Brisbane_Bus['stop_id'] = VIC_Brisbane_Bus['stop_id'].fillna('u')

In [ ]:
VIC_Brisbane_Bus[VIC_Brisbane_Bus.duplicated()].count()

In [ ]:
VIC_Brisbane_Bus.info()

In [ ]:
VIC_Brisbane_Bus['stop_id'] = VIC_Brisbane_Bus['stop_id'].fillna('Unknown')
VIC_Brisbane_Bus

In [ ]:
VIC_Brisbane_Bus = add_id_and_postcode_cols(VIC_Brisbane_Bus)
VIC_Brisbane_Bus.info()

In [ ]:
VIC_Brisbane_Bus = get_postcode(VIC_Brisbane_Bus)

In [ ]:
VIC_Brisbane_Bus.to_csv('dataset/finished/VIC_Brisbane_Bus.csv', index=False)

## SA_Adelaide_Metro

In [ ]:
SA_Adelaide_Metro = pd.read_csv('dataset/SA_Adelaide Metro station.csv')
SA_Adelaide_Metro.head(2)

In [ ]:
SA_Adelaide_Metro[SA_Adelaide_Metro.duplicated()].count()

In [ ]:
SA_Adelaide_Metro.info()

In [ ]:
SA_Adelaide_Metro = add_id_and_postcode_cols(SA_Adelaide_Metro)
SA_Adelaide_Metro.head(2)

In [ ]:
SA_Adelaide_Metro[SA_Adelaide_Metro['postcode'] == '']

In [ ]:
SA_Adelaide_Metro = get_postcode(SA_Adelaide_Metro)

In [ ]:
SA_Adelaide_Metro.to_csv('dataset/finished/SA_Adelaide_Metro.csv', index=False)

## VIC_Regional_Bus

In [ ]:
VIC_Regional_Bus = pd.read_csv('dataset/VIC_regional_bus_stop.csv')
VIC_Regional_Bus.head(2)

In [ ]:
VIC_Regional_Bus[VIC_Regional_Bus.duplicated()].count()

In [ ]:
VIC_Regional_Bus.info()

In [ ]:
VIC_Regional_Bus = VIC_Regional_Bus.rename(columns={'stop_long':'stop_lon'})

In [ ]:
VIC_Regional_Bus = add_id_and_postcode_cols(VIC_Regional_Bus)
VIC_Regional_Bus.head(2)

In [ ]:
VIC_Regional_Bus[VIC_Regional_Bus['postcode'] == '']

In [ ]:
VIC_Regional_Bus = get_postcode(VIC_Regional_Bus)

In [ ]:
VIC_Regional_Bus.to_csv('dataset/finished/VIC_Regional_Bus.csv', index=False)

## Tasmania_Metro_Hobart

In [ ]:
Tasmania_Metro_Hobart = pd.read_csv('dataset/Tasmania - Metro Hobart _GTFS v2.csv')
Tasmania_Metro_Hobart.head(2)

In [ ]:
Tasmania_Metro_Hobart[Tasmania_Metro_Hobart.duplicated()].count()

In [ ]:
Tasmania_Metro_Hobart.info()

In [ ]:
Tasmania_Metro_Hobart = add_id_and_postcode_cols(Tasmania_Metro_Hobart)
Tasmania_Metro_Hobart.head(2)

In [ ]:
Tasmania_Metro_Hobart[Tasmania_Metro_Hobart['postcode'] == '']

In [ ]:
Tasmania_Metro_Hobart = get_postcode(Tasmania_Metro_Hobart)

In [ ]:
Tasmania_Metro_Hobart.to_csv('dataset/finished/Tasmania_Metro_Hobart.csv', index=False)

## Tasmania_Metro_Launceston

In [ ]:
Tasmania_Metro_Launceston = pd.read_csv('dataset/Tasmania - Metro  Launceston _GTFS V2.csv')
Tasmania_Metro_Launceston.head(2)

In [ ]:
Tasmania_Metro_Launceston[Tasmania_Metro_Launceston.duplicated()].count()

In [ ]:
Tasmania_Metro_Launceston[Tasmania_Metro_Launceston.duplicated()]

In [ ]:
Tasmania_Metro_Launceston.info()

In [ ]:
Tasmania_Metro_Launceston = Tasmania_Metro_Launceston.iloc[:-4,:]

In [ ]:
Tasmania_Metro_Launceston.info()

In [ ]:
Tasmania_Metro_Launceston = add_id_and_postcode_cols(Tasmania_Metro_Launceston)
Tasmania_Metro_Launceston.head(2)

In [ ]:
Tasmania_Metro_Launceston[Tasmania_Metro_Launceston['postcode'] == '']

In [ ]:
Tasmania_Metro_Launceston = get_postcode(Tasmania_Metro_Launceston)

In [ ]:
Tasmania_Metro_Launceston.to_csv('dataset/finished/Tasmania_Metro_Launceston.csv', index=False)

## NT_Bus

In [ ]:
NT_Bus = pd.read_csv('dataset/NT_Bus.csv')
NT_Bus.head(2)

In [ ]:
NT_Bus[NT_Bus.duplicated()].count()

In [ ]:
NT_Bus.info()

In [ ]:
NT_Bus = add_id_and_postcode_cols(NT_Bus)
NT_Bus.info()

In [ ]:
NT_Bus = get_postcode(NT_Bus)

In [ ]:
NT_Bus.to_csv('dataset/finished/NT_Bus.csv', index=False)

## Tasmania_Metro_Burnie

In [ ]:
Tasmania_Metro_Burnie = pd.read_csv('dataset/Tasmania -Metro  Burnie.csv')
Tasmania_Metro_Burnie.head(2)

In [ ]:
Tasmania_Metro_Burnie[Tasmania_Metro_Burnie.duplicated()].count()

In [ ]:
Tasmania_Metro_Burnie.info()

In [ ]:
Tasmania_Metro_Burnie = add_id_and_postcode_cols(Tasmania_Metro_Burnie).iloc[:302,:]
Tasmania_Metro_Burnie.info()

In [ ]:
Tasmania_Metro_Burnie = get_postcode(Tasmania_Metro_Burnie)

In [ ]:
Tasmania_Metro_Burnie.to_csv('dataset/finished/Tasmania_Metro_Burnie.csv', index=False)

## SA_Bus_Shelter

In [ ]:
SA_Bus_Shelter = pd.read_csv('dataset/SA_BusShelters.csv')
SA_Bus_Shelter.head(2)

In [ ]:
SA_Bus_Shelter.info()

In [ ]:
SA_Bus_Shelter[SA_Bus_Shelter.duplicated()].count()

In [ ]:
for i in range(len(SA_Bus_Shelter)):
    coord = transformer.transform(SA_Bus_Shelter['stop_lon'][i], SA_Bus_Shelter['stop_lat'][i])
    SA_Bus_Shelter['stop_lat'][i] = coord[0]
    SA_Bus_Shelter['stop_lon'][i] = coord[1]
    

In [ ]:
SA_Bus_Shelter = add_id_and_postcode_cols(SA_Bus_Shelter)
SA_Bus_Shelter.info()

In [ ]:
SA_Bus_Shelter = get_postcode(SA_Bus_Shelter)

In [ ]:
SA_Bus_Shelter.to_csv('dataset/finished/SA_Bus_Shelter.csv', index=False)

## NSW_Station

In [ ]:
NSW_Station = pd.read_csv('dataset/NSW_Station.csv')
NSW_Station.head(2)

In [ ]:
NSW_Station[NSW_Station.duplicated()].count()

In [ ]:
NSW_Station.info()

In [ ]:
NSW_Station['stop_id'] = NSW_Station['stop_id'].fillna('Unknown')

In [ ]:
NSW_Station.rename(columns={'stop_long':'stop_lon'}, inplace=True)

In [ ]:
NSW_Station = add_id_and_postcode_cols(NSW_Station)
NSW_Station.info()

In [ ]:
NSW_Station[NSW_Station['postcode'] == '']

In [ ]:
NSW_Station = get_postcode(NSW_Station)

In [ ]:
NSW_Station.to_csv('dataset/finished/NSW_Station.csv', index=False)